In [23]:
from openpyxl import Workbook, load_workbook
from fuzzywuzzy import process, fuzz
import time
import pandas as pd
import numpy as np

In [24]:
from szp_funcs import print_df, path_to_data

In [25]:
path_to_dir = 'C:/Users/PetukhovMD/Desktop/Направления и категории/'
file = 'новые данные в выгрузках.xlsx'
pages = ['job_cat', 'job_direc', 'source']

In [26]:
categories = pd.read_excel(path_to_dir + file, sheet_name= pages[0])
direction = pd.read_excel(path_to_dir + file, sheet_name= pages[1])
source = pd.read_excel(path_to_dir + file, sheet_name= pages[2])

In [27]:
jobs_source = source['job'].unique().tolist()
jobs_no_direction = source[source['direc'].isna()]['job'].unique().tolist()
directions_source = source['direc'].unique().tolist()

In [28]:
def matcher(row, look_up, name):
    ans = process.extractOne(row[name], look_up, scorer=fuzz.WRatio, score_cutoff=60)
    if ans:
        return ans[0]
    return ans

In [29]:
categories['real_job'] = categories.apply(lambda row: matcher(row, jobs_source, 'job'), axis=1)
direction['real_job'] = direction.apply(lambda row: matcher(row, jobs_source, 'job'), axis=1)

In [30]:
out_cat_by_job = categories.drop('job', axis=1).groupby('real_job').agg({
    'job_cat': 'unique'
}).reset_index()

In [31]:
def to_good_str(row, name):
    s = ''
    for i in  row[name]:
        s += str(i) + '&'
    return s[:-1]

In [32]:
out_cat_by_job['job_cat_str'] = out_cat_by_job.apply(lambda row: to_good_str(row, 'job_cat'), axis=1)

In [33]:
print_df(out_cat_by_job.drop('job_cat', axis=1), path_to_dir + 'категории по должностям.xlsx')

In [34]:
direction[(direction['real_job'].isin(jobs_no_direction)) & (direction['job_direc'].isna() == False)]

,job,job_direc,real_job
68,Воспитатель,группы продленного дня,Воспитатель
80,Воспитатель,группы кратковременного пребывания,Воспитатель
81,Помощник воспитателя,семейного детского сада,Помощник воспитателя
85,Воспитатель,семейного детского сада,Воспитатель
93,Начальник отдела,по организации учебно-воспитательной деятельности,Начальник отдела
...,...,...,...
569,Младший воспитатель,группы ОВЗ,Младший воспитатель
600,Ведущий специалист,приёмной комиссии,Ведущий специалист
643,Специалист,офицер кадетского класса,Специалист
646,Начальник группы,в сфере ИКТ,Начальник группы


In [35]:
print_df(direction[direction['job_direc'].isin(directions_source + [np.nan]) == False][['real_job', 'job_direc']], path_to_dir + 'некорректные направления по должностям')

In [36]:
categories

,job,job_cat,real_job
0,Преподаватель,Педагогические работники дошкольных образовате...,Преподаватель
1,Библиотекарь,Прочий персонал,Библиотекарь
2,Специалист,Прочий персонал,Специалист
3,Методист,Педагогические работники образовательных учреж...,Методист
4,Начальник отдела,Прочий персонал,Начальник отдела
...,...,...,...
888,Секретарь по учебной части,NaN,Секретарь учебной части
889,Начальник отдела,Административно-управленческий,Начальник отдела
890,Руководитель,Руководитель организации,Руководитель
891,Консультант,NaN,Консультант


In [37]:
direction

,job,job_direc,real_job
0,Преподаватель,NaN,Преподаватель
1,Библиотекарь,NaN,Библиотекарь
2,Специалист,NaN,Специалист
3,Методист,NaN,Методист
4,Начальник отдела,NaN,Начальник отдела
...,...,...,...
647,Начальник АХО,NaN,Начальник
648,Переводчик-дактилолог (сурдопереводчик),NaN,Переводчик-дактилолог
649,Секретарь по учебной части,NaN,Секретарь учебной части
650,Начальник сектора,правового сопровождения,Начальник сектора
